In [1]:
import pychor
import galois

# Multiplication Triples

We've seen how to use additive secret sharing to build protocols for summation, by using the additive homomorphism property of additive shares.

What if we want to do multiplication? Unfortunately, additive shares don't have a multiplicative homomorphism!

Fortunately, there's a commonly-used trick for performing multiplication on additive secret shares that leverages a construction called a *multiplication triple*. The idea is due to [Don Beaver](https://sites.google.com/view/donbeaver/home) {cite}`beaver1991efficient`; multiplication triples are often called *Beaver triples*, and the approach is sometimes called *Beaver's trick*.

````{admonition} Further reading: Multiplication Triples
:class: seealso

For more information on the GMW protocol, see **Section 3.4 of [Pragmatic MPC](https://securecomputation.org/)**.
````

## What is a Multiplication Triple?

A multiplication triple is values $a, b, c$ such that $a * b = c$, but with an important property:

- $a$, $b$, and $c$ are *secret shared* among the parties
- *No party* knows the actual values of $a$, $b$, and $c$ - each party only knows their own share

So, in the two-party setting, a multiplication triple is actually represented by six secret shares:

- P1 knows $a_1$, $b_1$, and $c_1$
- P2 knows $a_2$, $b_2$, and $c_2$
- $(a_1 + a_2) * (b_1 + b_2) = c_1 + c_2$
- Neither P1 nor P2 knows $a$, $b$, or $c$

Multiplication triples can be used to perform multiplication of arbitrary additive shares, as we'll see later in this chapter.

## Generating Multiplication Triples using a Dealer

No party is supposed to know the values $a$, $b$, and $c$ - so how do we generate shares of them, without revealing them to any of the parties?

This actually turns out to be quite a difficult problem, and is one of the major sticking points in building high-performance MPC protocols. It's fundamentally a circular problem - we can't generate a triple that is unknown to all parties without performing some multiplication, but we can't perform multiplication without having access to some triples! We'll study some options for doing this later in this book.

For now, we'll describe the ideal functionality for generating multiplication triples, and defer implementing a protocol that realizes the functionality until later. Our ideal functionality will use the trusted "functionality party" to generate the multiplication triple and send secret shares of the triple to the participating parties. In practice, a secure protocol realizing the functionality would be used instead, unless an actual trusted third party exists who can generate the triples.

````{admonition} Ideal functionality: generate multiplication triple
:class: tip

The ideal functionality for generating a new multiplication triple for the parties P1 and P2 has two steps:
1. The "functionality party" FGen generates the triple $a$, $b$, $c$
2. FGen secret shares $a$, $b$, and $c$, and sends shares to P1 and P2
````

We can implement this functionality as follows:

In [5]:
p1 = pychor.Party('p1')
p2 = pychor.Party('p2')
Fgen = pychor.Party('Fgen')
GF = galois.GF(17)

@pychor.local_function
def share(x):
    s1 = GF.Random()
    s2 = GF(x) - s1
    return s1, s2

def deal_shares(x):
    s1, s2 = share(x).untup(2)
    s1.send(Fgen, p1)
    s2.send(Fgen, p2)
    return s1, s2

def functionality_gen_triple():
    # Step 1: generate a, b, c
    a = Fgen.constant(GF.Random())
    b = Fgen.constant(GF.Random())
    c = a * b

    # Step 2: secret share a, b, c
    a1, a2 = deal_shares(a)
    b1, b2 = deal_shares(b)
    c1, c2 = deal_shares(c)
    return (a1, a2), (b1, b2), (c1, c2)

We can use the `functionality_gen_triple` ideal functionality to generate new multiplication triples, like this:

In [6]:
with pychor.LocalBackend():
    # Generate and deal the triple
    t = functionality_gen_triple()
    print('Shared triple:', t)

    # Pattern match on the structure of the triple to extract the shares
    (a1, a2), (b1, b2), (c1, c2) = t

    print('Reconstructed a:', a1 + a2)
    print('Reconstructed b:', b1 + b2)
    print('Reconstructed c:', c1 + c2)
    print('Reconstructed a*b:', (a1+a2)*(b1+b2))

Shared triple: ((5@{p1, Fgen}, 16@{p2, Fgen}), (6@{p1, Fgen}, 13@{p2, Fgen}), (14@{p1, Fgen}, 11@{p2, Fgen}))
Reconstructed a: 4@{Fgen}
Reconstructed b: 2@{Fgen}
Reconstructed c: 8@{Fgen}
Reconstructed a*b: 8@{Fgen}


As expected, the shares of the triple are such that $(a_1 + a_2) * (b_1 + b_2) = c_1 + c_2$, except that all arithmetic is done $\mod p$. Each time we run the program, we'll get a new multiplication triple.

## Multiplying Additive Shares using Multiplication Triples

How do we use a multiplication triple to actually perform multiplication? We can use Beaver's trick to perform the multiplication, consuming one multiplication triple in the process. The following protocol performs the multiplication, and is due to Beaver {cite}`beaver1991efficient`.

````{admonition} Protocol: multiplication of secret-shared values using a multiplication triple
:class: note

Setup:
- We want to multiply $x$ and $y$
- P1 has shares $x_1$ and $y_1$
- P2 has shares $x_2$ and $y_2$
- P1 has shares $a_1$, $b_1$, and $c_1$ of a multiplication triple
- P2 has shares $a_2$, $b_2$, and $c_2$ of the same multiplication triple

The parties P1 and P2 follow the following steps:
1. P1 computes $x_1 - a_1$ and sends the result to P2
2. P2 computes $x_2 - a_2$ and sends the result to P1
3. P1 and P2 both compute $d = (x_1 - a_1) + (x_2 - a_2) = (x_1 + x_2) - (a_1 + a_2) = x - a$
4. P1 computes $y_1 - b_1$ and sends the result to P2
5. P2 computes $y_2 - b_2$ and sends the result to P1
6. P1 and P2 both compute $e = (y_1 - b_1) + (y_2 - b_2) = (y_1 + y_2) - (b_1 + b_2) = y - b$
7. P1 computes $r_1 = d*e + d*b_1 + e*a_1 + c_1$
8. P2 computes $r_2 = 0 + d*b_2 + e*a_2 + c_2$

At the end of the protocol, $r_1$ and $r_2$ are additive shares of $x*y$.
````

**Is it secure?** In steps 1, 2, 4, and 5, each party sends results computed from their shares to the other party. Doesn't this violate security? Fortunately not - the shares of $a$ and $b$ are not known to the other party, and can't be inferred from the received values, since neither party knows $a$ or $b$ either! This is why it's so important that the values of $a$, $b$, and $c$ be unknown to the parties - these values effectively *mask* the shares being sent during multiplication.

This use of the multiplication triple for masking means that **each multiplication triple can only be used once**, to perform a single multiplication. If a triple is used more than once, then correlations between input values may enable a participant to infer the value of $a$, $b$, or $c$, and thus learn the values of the input shares. **Using a multiplication triple to perform a multiplication "consumes" it** and it can't be used again.

**Is it correct?** Consider the following identity:

$$
\begin{align*}
x*y =& (x - a + a)(y - b + b)\\
=& (d+a)(e+b)\\
=& de + db + ae + ab\\
=& de + db + ae + c
\end{align*}
$$

The shares $r_1$ and $r_2$ computed at the end of the protocol are exactly additive shares of this value!

We can implement this protocol using a choreographic function called `protocol_mult` whose inputs are secret-shared values `x` and `y` and a multiplication triple:

In [7]:
def protocol_mult(x, y, triple):
    x1, x2 = x
    y1, y2 = y
    (a1, a2), (b1, b2), (c1, c2) = triple

    # Step 1. P1 computes d_1 = x_1 - a_1 and sends the result to P2
    d1 = x1 - a1
    d1.send(p1, p2)

    # Step 2. P2 computes d_2 = x_2 - a_2 and sends the result to P1
    d2 = x2 - a2
    d2.send(p2, p1)

    # Step 3: P1 and P2 both compute $d = d_1 + d_2 = x - a$
    d = d1 + d2

    # Step 4. P1 computes e_1 = y_1 - b_1 and sends the result to P2
    e1 = y1 - b1
    e1.send(p1, p2)

    # Step 5. P2 computes e_2 = y_2 - b_2 and sends the result to P1    
    e2 = y2 - b2
    e2.send(p2, p1)

    # Step 6. P1 and P2 both compute $e = e_1 + e_2 = y - b$
    e = e1 + e2

    # Step 7. P1 computes r_1 = d*e + d*b_1 + e*a_1 + c_1
    r_1 = d * e + d * b1 + e * a1 + c1

    # Step 8. P2 computes r_2 = 0 + d*b_2 + e*a_2 + c_2
    r_2 = d * b2 + e * a2 + c2

    return r_1, r_2

We can apply the protocol by passing in secret-shared values (like the ones we constructed in Chapter 2) and a multiplication triple generated by the functionality we wrote earlier.

In [8]:
with pychor.LocalBackend():
    # P1 knows the input x, and P2 knows the input y
    x_input = 3@p1
    y_input = 4@p2

    # Create secret shares of the inputs
    x1, x2 = share(x_input).untup(2)
    y1, y2 = share(y_input).untup(2)
    x2.send(p1, p2)
    y1.send(p2, p1)

    # Generate a multiplication triple
    triple = functionality_gen_triple()

    # Perform the multiplication
    r1, r2 = protocol_mult((x1, x2), (y1, y2), triple)

    # Broadcast results and print the product
    r1.send(p1, p2)
    r2.send(p2, p1)
    print('Product:', r1 + r2)

Product: 12@{p2, p1}


As expected, the product of 3 and 4 is 12. As in the summation protocol, we need to be careful that the size of the finite field used in the protocol is large enough to hold the product, or it will wrap around and we'll get the wrong answer.

This protocol is quite a bit more complicated than the summation protocol we built earlier. It requires the dealer to communicate the multiplication triple to the parties, and it also requires the parties to communicate two field elements to each other during the multiplication step. The sequence diagram below describes an example execution of the protocol:

```{mermaid}
sequenceDiagram
participant p1
participant p2
participant Fgen
Note right of Fgen: Secret share inputs
p1 ->> p2 : 14
p2 ->> p1 : 13
Note right of Fgen: Generate multiplication triple
dealer ->> p1 : 16
dealer ->> p2 : 8
dealer ->> p1 : 7
dealer ->> p2 : 3
dealer ->> p1 : 0
dealer ->> p2 : 2
Note right of Fgen: Perform multiplication
p1 ->> p2 : 7
p2 ->> p1 : 6
p1 ->> p2 : 6
p2 ->> p1 : 5
Note right of Fgen: Reconstruct product
p1 ->> p2 : 2
p2 ->> p1 : 10
```

## Why use Multiplication Triples?

Using multiplication triples to do multiplication of additive secret shares has a major issue: it requires the presence of a trusted dealer, or the use of a more complicated (and expensive) protocol to generate the triples. A trusted dealer generally obviates the need for a secure protocol in the first place, since the dealer could simply perform the multiplication themselves! So, what's the point of using multiplication triples?

As we'll continue to see throughout this book: *secure multiplication is hard*. There is no magical solution that provides a simple, convenient, efficient approach for multiplication (as we have for addition, via the additive homomorphism of secret shares). Alternatives to multiplication triples make different tradeoffs, but none of them provides a solution that is better in every way.

The multiplication triple approach differs from many of the alternatives by separating the triple generation step from the multiplication itself. This has several advantages:
- It makes protocols modular: a protocol that uses multiplication triples to perform multiplication can choose from many different approaches for *generating* the triples, and any of them will work. We'll see examples of these later in the book.
- It allows generating triples in parallel, since triples are independent from each other.
- It enables performing the expensive triple generation portion as a pre-processing step, since the triples don't depend on the actual input to the protocol. This is often referred to as an "offline" step that the parties can perform ahead of time. In settings where reacting quickly to new inputs is important, this can be a huge benefit.
- It makes the multiplication step simple: the approach presented in this chapter is perhaps the simplest protocol for multiplying secret shares.
